# F1 Race Prediction
#### **Project**: Predicting the results of F1 races based on historical data
#### **Context**: The growing F1 industry gets more competitive every year, with performance being more calculated than ever. Would it be possible to predict the results?

## 1. Introduction/Business Problem

F1 prediction - does engine play a major impact?

In [10]:
import pandas as pd
import numpy as np

circuits_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\circuits.csv')
constructor_results_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\constructor_results.csv')
constructor_standings_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\constructor_standings.csv')
constructors_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\constructors.csv')
driver_standigs_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\driver_standings.csv')
drivers_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\drivers.csv')
lap_times_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\lap_times.csv')
pit_stops_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\pit_stops.csv')
qualifying_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\qualifying.csv')
races_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\races.csv')
results_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\results.csv')
seasons_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\seasons.csv')
status_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\status.csv')
weather_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\data\weather_1950_to_2020.csv')

### Merging Races, Circuits and Weather


In [11]:
# Adding a circuit reference to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['circuitRef'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Circuit Reference'] = circuits_in_races.replace(circuits_dict)

# Adding a circuit location to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['location'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Location'] = circuits_in_races.replace(circuits_dict)

# Adding a circuit country to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['country'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Country'] = circuits_in_races.replace(circuits_dict)

# Adding a circuit latitude to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['lat'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Latitude'] = circuits_in_races.replace(circuits_dict)

# Adding a circuit longitude to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['lng'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Longitude'] = circuits_in_races.replace(circuits_dict)

# Adding a circuit longitude to each corresponding circuit Id
races_df['Dry'] = weather_df['Dry']
races_df['Wet'] = weather_df['Wet']
races_df['Drizzle'] = weather_df['Drizzle']
races_df['Rain'] = weather_df['Rain']

# Droping irrelevant columns
races_df = races_df.drop(columns=['raceId','time','circuitId'])

races_df = races_df[['year', 'date', 'round', 'name', 'Circuit Reference', 'Location', 'Country', 'Latitude','Longitude','Dry','Wet','Drizzle','Rain']]
races_df.rename(columns={'round': 'Round','year': 'Year', 'name': 'Name', 'date': 'Date'})

,Year,Date,Round,Name,Circuit Reference,Location,Country,Latitude,Longitude,Dry,Wet,Drizzle,Rain
0,2009,2009-03-29,1,Australian Grand Prix,albert_park,Melbourne,Australia,-37.84970,144.96800,1.0,0.0,0.0,0.0
1,2009,2009-04-05,2,Malaysian Grand Prix,sepang,Kuala Lumpur,Malaysia,2.76083,101.73800,1.0,0.0,0.0,0.0
2,2009,2009-04-19,3,Chinese Grand Prix,shanghai,Shanghai,China,31.33890,121.22000,0.0,0.0,1.0,0.0
3,2009,2009-04-26,4,Bahrain Grand Prix,bahrain,Sakhir,Bahrain,26.03250,50.51060,1.0,0.0,0.0,0.0
4,2009,2009-05-10,5,Spanish Grand Prix,catalunya,Montmeló,Spain,41.57000,2.26111,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,2020,2020-11-01,13,Emilia Romagna Grand Prix,imola,Imola,Italy,44.34390,11.71670,0.0,0.0,0.0,0.0
1031,2020,2020-11-15,14,Turkish Grand Prix,istanbul,Istanbul,Turkey,40.95170,29.40500,0.0,0.0,0.0,0.0
1032,2020,2020-11-29,15,Bahrain Grand Prix,bahrain,Sakhir,Bahrain,26.03250,50.51060,0.0,0.0,0.0,0.0
1033,2020,2020-12-06,16,Sakhir Grand Prix,bahrain,Sakhir,Bahrain,26.03250,50.51060,0.0,0.0,0.0,0.0


### Results